<a href="https://colab.research.google.com/github/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/blob/main/Code/New_Notebook_Blancpain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DMML TEAM BLANCPAIN

##Import some librairies

In [28]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import collections  as mc
%load_ext autoreload
%autoreload 2
import pandas as pd 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
sns.set_style("white")
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.utils.multiclass import unique_labels

%matplotlib inline
sns.set_style("dark")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


##Data Importation##

In [2]:
df_train = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/training_data.csv')

df_test = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/test_data.csv')

df_sample = pd.read_csv('https://raw.githubusercontent.com/sfrancey/Real-or-Not-NLP-with-Disaster-Tweets_Team_Blancpain/main/Data/sample_submission.csv')

##Data visualization##

###Training set

In [3]:
df_train.head()

,id,keyword,location,text,target
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0
1,853,bioterror,NaN,#world FedEx no longer to transport bioterror ...,0
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1


In [4]:
print("There are {0} rows and {1} columns in the train dataset.".format(df_train.shape[0],df_train.shape[1]))

There are 6471 rows and 5 columns in the train dataset.


###Test Set

In [5]:
df_test.head()

,id,keyword,location,text
0,9972,tsunami,NaN,Crptotech tsunami and banks.\n http://t.co/KHz...
1,9865,traumatised,"Portsmouth, UK",I'm that traumatised that I can't even spell p...
2,1937,burning%20buildings,NaN,@foxnewsvideo @AIIAmericanGirI @ANHQDC So ... ...
3,3560,desolate,NaN,Me watching Law &amp; Order (IB: @sauldale305)...
4,2731,crushed,bahstun/porta reeko,Papi absolutely crushed that ball


In [6]:
print("There are {0} rows and {1} columns in the test dataset.".format(df_test.shape[0],df_test.shape[1]))

There are 1142 rows and 4 columns in the test dataset.


###Drop Null values

In [7]:
df_train.fillna("Unknown", inplace = True)
df_test.fillna("Unknown", inplace = True)

###Base Rate 

In [8]:
base_rate = max(len(df_train[df_train["target"] == 0]) / len(df_train), len(df_train[df_train["target"] == 1]) / len(df_train))
print("The base rate for this problem is :", base_rate)

The base rate for this problem is : 0.5719363313243703


##Processing

In [9]:
df_train["keyword_text"] = df_train['keyword'] + ' ' + df_train['text'] 
df_train["keyword_text"] = df_train["keyword_text"].astype(str)
df_test["keyword_text"] = df_test['keyword'] + ' ' + df_test['text'] 
df_test["keyword_text"] = df_test["keyword_text"].astype(str)

In [10]:
df_train

,id,keyword,location,text,target,keyword_text
0,3738,destroyed,USA,Black Eye 9: A space battle occurred at Star O...,0,destroyed Black Eye 9: A space battle occurred...
1,853,bioterror,Unknown,#world FedEx no longer to transport bioterror ...,0,bioterror #world FedEx no longer to transport ...
2,10540,windstorm,"Palm Beach County, FL",Reality Training: Train falls off elevated tra...,1,windstorm Reality Training: Train falls off el...
3,5988,hazardous,USA,#Taiwan Grace: expect that large rocks trees m...,1,hazardous #Taiwan Grace: expect that large roc...
4,6328,hostage,Australia,New ISIS Video: ISIS Threatens to Behead Croat...,1,hostage New ISIS Video: ISIS Threatens to Behe...
...,...,...,...,...,...,...
6466,4377,earthquake,ARGENTINA,#Earthquake #Sismo M 1.9 - 15km E of Anchorage...,1,earthquake #Earthquake #Sismo M 1.9 - 15km E o...
6467,3408,derail,Unknown,@EmiiliexIrwin Totally agree.She is 23 and kno...,0,derail @EmiiliexIrwin Totally agree.She is 23 ...
6468,9794,trapped,Unknown,Hollywood Movie About Trapped Miners Released ...,1,trapped Hollywood Movie About Trapped Miners R...
6469,10344,weapons,Beirut/Toronto,Friendly reminder that the only country to eve...,1,weapons Friendly reminder that the only countr...


In [11]:
df_train_train, df_train_test = train_test_split(df_train, test_size=0.2, random_state = 50)

In [35]:
y_train_train = df_train_train[["target"]]
y_train_test = df_train_test[["target"]]

###With "Keyword" feature

In [36]:
df_train_train.reset_index(drop=True, inplace=True)
df_train_test.reset_index(drop=True, inplace=True)

In [39]:
label = LabelEncoder()
label_keyword_train = pd.Series(label.fit_transform(df_train_train["keyword"]), name = "keyword")
df_train_test["keyword"] = pd.Series(label.fit_transform(df_train_test["keyword"]), name = "keyword")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [41]:
df_label_keyword_train = pd.DataFrame(label_keyword_train)

In [44]:
regression_CV = LogisticRegressionCV(solver='lbfgs', cv=5, max_iter=1000, random_state=50)
regression_CV.fit(df_label_keyword_train, y_train_train.values.ravel())
y_pred = regression_CV.predict(df_label_keyword_train)